# Explore simim drivers/outputs

In [ ]:
import glob
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from geopandas.plotting import plot_polygon_collection

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
glob.glob("../simim/data/output/simim_*.csv")

In [ ]:
lads = gpd.read_file('../simim/data/cache/Local_Authority_Districts_December_2016_Ultra_Generalised_Clipped_Boundaries_in_Great_Britain.shp')

In [ ]:
lads.plot()

In [ ]:
arc_lads = pd.read_csv('../simim/data/scenarios/camkox_lads.csv')
arc_lads.head(3)

## Read baseline/scenario data

In [ ]:
import simim.data_apis

In [ ]:
os.chdir("../simim")

In [ ]:
simim_data = simim.data_apis.Instance({
    "coverage": "GB", 
    "cache_dir": "./data/cache", 
    "output_dir": "./data/output",
    "model_type": "none",
    "base_projection": "ppp",
    "scenario": "none",
    "attractors": []
})

In [ ]:
dfs = []
for year in range(2015, 2051):
    df = simim_data.get_households(year, lads.lad16cd.unique())
    dfs.append(df)
households = pd.concat(dfs, sort=False).rename(columns={"PROJECTED_YEAR_NAME": "YEAR"})
households.head()

In [ ]:
households[households.GEOGRAPHY_CODE=='E07000004'].head()

In [ ]:
df_emp = pd.read_csv("../simim/data/arc/arc_employment__baseline.csv")
df_gva = pd.read_csv("../simim/data/arc/arc_gva__baseline.csv")
df_dwl = pd.read_csv("../simim/data/arc/arc_dwellings__baseline.csv")

# merge to single dataframe
df = df_gva \
  .merge(df_emp, on=["timestep", "lad_uk_2016"], how="left") \
  .merge(df_dwl, on=["timestep", "lad_uk_2016"], how="left")

baseline = df.reset_index().rename(columns={
    "timestep": "YEAR", 
    "lad_uk_2016": "GEOGRAPHY_CODE", 
    "employment": "JOBS_baseline", 
    "gva": "GVA_baseline", 
    "gva_per_sector": "GVA_baseline",
    "dwellings": "DWELLINGS_baseline"
})[[
 "YEAR", "GEOGRAPHY_CODE", "JOBS_baseline", "GVA_baseline", "DWELLINGS_baseline"
]].merge(
  households, on=["GEOGRAPHY_CODE", "YEAR"]
).rename(columns={"HOUSEHOLDS": "HOUSEHOLDS_baseline"})
baseline["GVA_baseline"] = baseline["GVA_baseline"].round(6)
# convert from 1000s jobs to jobs
baseline["JOBS_baseline"] = (baseline["JOBS_baseline"] * 1000).round().astype(int)
baseline.head()

In [ ]:
baseline[baseline.GEOGRAPHY_CODE.isin(arc_lads.geo_code)] \
  .groupby(["YEAR"]).sum() \
  [['DWELLINGS_baseline', 	'HOUSEHOLDS_baseline']] \
  .plot()

In [ ]:
dataset = baseline
for key in ('0-unplanned', '1-new-cities', '2-expansion'):
    if key == "3-new-cities23":
        econ_key = "1-new-cities"
    elif key == "4-expansion23":
        econ_key = "2-expansion"
    else:
        econ_key = key
    df_gva = pd.read_csv("../simim/data/arc/arc_gva__{}.csv".format(econ_key))
    df_emp = pd.read_csv("../simim/data/arc/arc_employment__{}.csv".format(econ_key))
    df_dwl = pd.read_csv("../simim/data/arc/arc_dwellings__{}.csv".format(key))
    
    gva_key = "GVA_{}".format(key)
    jobs_key = "JOBS_{}".format(key)
    households_key = "HOUSEHOLDS_{}".format(key)

    # merge to single dataframe
    scenario = df_gva \
    .merge(df_emp, on=["timestep", "lad_uk_2016"], how="left") \
    .merge(df_dwl, on=["timestep", "lad_uk_2016"], how="left") \
    .drop("lad16nm", axis=1) \
    .rename(columns={"timestep": "YEAR", "lad_uk_2016": "GEOGRAPHY_CODE", "gva_per_sector": gva_key,
                     "employment": jobs_key,  "dwellings": households_key})

    
    dataset = dataset.merge(scenario, on=["YEAR", "GEOGRAPHY_CODE"])
    
    
    dataset[households_key] = dataset[households_key] - dataset.DWELLINGS_baseline + dataset.HOUSEHOLDS_baseline
    
    dataset[gva_key] = dataset[gva_key].round(6)
    dataset[jobs_key] = (dataset[jobs_key] * 1000).round().astype(int)  # convert from 1000s jobs to jobs
    dataset[households_key] = dataset[households_key].round().astype(int)

    
    
dataset.head()

In [ ]:
dataset[dataset.GEOGRAPHY_CODE.isin(arc_lads.geo_code)]

In [ ]:
access = pd.read_csv('../simim/data/access_baseline_road_rail.csv').rename(columns={
    "ORIGIN_ZONE_CODE": "O_GEOGRAPHY_CODE", 
    "DESTINATION_ZONE_CODE": "D_GEOGRAPHY_CODE",
    "GENERALISED_TRAVEL_COST": "ACCESSIBILITY"
})

## Read outputs

In [ ]:
output_path = '../simim/data/output/'
outputs = {
    '0-unplanned': 'simim_gravity_ppp_scenario0-unplanned__gjh_D_HOUSEHOLDS-D_JOBS-D_GVA_EX_LONDON.csv',
    '1-new-cities': 'simim_gravity_ppp_scenario1-new-cities__gjh_D_HOUSEHOLDS-D_JOBS-D_GVA_EX_LONDON.csv',
    '2-expansion': 'simim_gravity_ppp_scenario2-expansion__gjh_D_HOUSEHOLDS-D_JOBS-D_GVA_EX_LONDON.csv',
    
#     '0-unplanned-h': 'simim_gravity_ppp_scenario0-unplanned__h_D_HOUSEHOLDS.csv',
#     '1-new-cities-h': 'simim_gravity_ppp_scenario1-new-cities__h_D_HOUSEHOLDS.csv',
#     '2-expansion-h': 'simim_gravity_ppp_scenario2-expansion__h_D_HOUSEHOLDS.csv',
    
#     '0-unplanned-odb': 'simim_gravity_ppp_scenario0-unplanned__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON.csv',
#     '1-new-cities-odb': 'simim_gravity_ppp_scenario1-new-cities__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON.csv',
#     '2-expansion-odb': 'simim_gravity_ppp_scenario2-expansion__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON.csv',
    
    '0-unplanned-od1': 'simim_gravity_ppp_scenario0-unplanned__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON__od_rail_b1.csv',
    '1-new-cities-od1': 'simim_gravity_ppp_scenario1-new-cities__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON__od_rail_b1.csv',
    '2-expansion-od1': 'simim_gravity_ppp_scenario2-expansion__gjh_D_HOUSEHOLDS-D_JOBS_ACCESSIBILITY-D_GVA_EX_LONDON__od_rail_b1.csv',
}
    
for key, filename in outputs.items():
    snpp_key = "PEOPLE_SNPP_{}".format(key)
    pp_key = "PEOPLE_{}".format(key)
    d_key = "RELATIVE_DELTA_{}".format(key)
    output = pd.read_csv(os.path.join(output_path, filename)) \
      .rename(columns={'PROJECTED_YEAR_NAME': 'YEAR', 'PEOPLE_SNPP': snpp_key, 'PEOPLE': pp_key, 'RELATIVE_DELTA': d_key})
    
    dataset = dataset.merge(output, on=["YEAR", "GEOGRAPHY_CODE"])

In [ ]:
dataset.columns

In [ ]:
def access_weighted_sum(dataset, colname, access_colname):
    new_colname = "D_{}_{}".format(colname, access_colname)
    # access to x[o] for each o,d 
    dataset[new_colname] = dataset["O_" + colname] * dataset[access_colname]
    # sum over o - grouping by d
    wsum = dataset[["D_GEOGRAPHY_CODE", new_colname]].groupby("D_GEOGRAPHY_CODE").sum().reset_index()

    # merge back
    dataset = dataset.merge(wsum, on="D_GEOGRAPHY_CODE") \
        .drop(new_colname + "_x", axis=1) \
        .rename({new_colname + "_y": new_colname}, axis=1)
    return dataset

In [ ]:
year = 2050

def add_access(df, access, year, keys):
    o_rename_columns = {}
    d_rename_columns = {}
    base_keys = ('baseline', '0-unplanned', '1-new-cities', '2-expansion')
    for key in keys:
        for var in ('PEOPLE', 'PEOPLE_SNPP', 'RELATIVE_DELTA'):
            o_rename_columns["{}_{}".format(var, key)] = "O_{}_{}".format(var, key)
            d_rename_columns["{}_{}".format(var, key)] = "D_{}_{}".format(var, key)
    for key in base_keys:
        for var in ('GVA', 'JOBS', 'HOUSEHOLDS'):
            o_rename_columns["{}_{}".format(var, key)] = "O_{}_{}".format(var, key)
            d_rename_columns["{}_{}".format(var, key)] = "D_{}_{}".format(var, key)
    
    
    dataset = access \
    .merge(
        df[df.YEAR == year].drop("YEAR", axis=1), 
        left_on="O_GEOGRAPHY_CODE", right_on="GEOGRAPHY_CODE"
    ) \
    .drop("GEOGRAPHY_CODE", axis=1) \
    .rename(columns=o_rename_columns) \
    .merge(
        df[df.YEAR == year].drop("YEAR", axis=1), 
        left_on="D_GEOGRAPHY_CODE", right_on="GEOGRAPHY_CODE"
    ) \
    .drop("GEOGRAPHY_CODE", axis=1) \
    .rename(columns=d_rename_columns)
    
    for key in base_keys:
        dataset = access_weighted_sum(dataset, "JOBS_{}".format(key), "ACCESSIBILITY")

    return dataset
        
with_access = add_access(dataset, access, 2050, ['baseline'] + list(outputs.keys()))
with_access.head()

In [ ]:
with_access.columns

In [ ]:
# keep only columns starting with D
d_columns = list(with_access.columns[with_access.columns.str.startswith("D")])
rename_columns = { 
    key: key.replace("D_", "") for key in d_columns
}
d_data = with_access[with_access.columns[with_access.columns.str.startswith("D")]] \
  .drop_duplicates() \
  .rename(columns=rename_columns)
d_data.head()

In [ ]:
d_data.columns

In [ ]:
eval_geo = lads.merge(d_data, left_on="lad16cd", right_on="GEOGRAPHY_CODE")
eval_geo.head()

In [ ]:
eval_geo["JOBS_DENSITY_baseline"] = eval_geo.JOBS_baseline / eval_geo.st_areasha
eval_geo["LOG_JOBS_DENSITY_baseline"] = np.log(eval_geo.JOBS_baseline / eval_geo.st_areasha)

In [ ]:
eval_geo.JOBS_baseline.hist(bins=100)

In [ ]:
eval_geo.plot(column="LOG_JOBS_DENSITY_baseline")

In [ ]:
eval_geo["LOG_JOBS_ACCESS_DENSITY_baseline"] = np.log(eval_geo.JOBS_baseline_ACCESSIBILITY / eval_geo.st_areasha)
eval_geo["JOBS_ACCESS_DENSITY_baseline"] = eval_geo.JOBS_baseline_ACCESSIBILITY / eval_geo.st_areasha

In [ ]:
eval_geo.plot(column="LOG_JOBS_ACCESS_DENSITY_baseline")

In [ ]:
eval_geo["HOUSEHOLDS_DIFF"] = eval_geo['HOUSEHOLDS_2-expansion'] - eval_geo.HOUSEHOLDS_baseline
eval_geo[eval_geo.HOUSEHOLDS_DIFF < -1]

In [ ]:
ax = eval_geo.plot(column="HOUSEHOLDS_DIFF")

ax.set_aspect('equal')
ax.set_xlim([380000, 670000])
ax.set_ylim([70000, 350000])

ax

In [ ]:
eval_geo["JOBS_DIFF"] = eval_geo['JOBS_2-expansion'] - eval_geo.JOBS_baseline
eval_geo[eval_geo.JOBS_DIFF > 0]

In [ ]:
eval_geo["GVA_DIFF"] = eval_geo['GVA_2-expansion'] - eval_geo.GVA_baseline
eval_geo[eval_geo.GVA_DIFF > 0]

In [ ]:
eval_geo.columns

In [ ]:
eval_geo["POPD"] = eval_geo['PEOPLE_2-expansion'] / eval_geo.st_areasha
eval_geo["POPD_SNPP"] = eval_geo['PEOPLE_SNPP_2-expansion'] / eval_geo.st_areasha
eval_geo["POPD_DIFF"] = eval_geo.POPD - eval_geo.POPD_SNPP

In [ ]:
def diff_plot(df, column):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), sharex=False, sharey=False)

    ax.set_aspect('equal')
    ax.set_xlim([120000, 670000])
    ax.set_ylim([0, 550000])
    ax.set_facecolor('xkcd:cerulean')

    net_out = df[df[column] <= 0.0]
    net_in = df[df[column] > 0.0]

    plot_polygon_collection(
        ax, 
        net_in['geometry'],
        clim=(0, np.max(net_in.POPD_DIFF)), 
        cmap="Reds",
        values=np.abs(net_in.POPD_DIFF)
    )
    plot_polygon_collection(
        ax, 
        net_out['geometry'],
        clim=(0, np.max(np.abs(net_out[column]))), 
        cmap="Blues",
        values=np.abs(net_out[column])
    )
    return ax

diff_plot(eval_geo, 'POPD_DIFF')

In [ ]:
totals = dataset[dataset.GEOGRAPHY_CODE.isin(arc_lads.geo_code) & dataset.YEAR.isin([2015, 2050])] \
  .groupby("YEAR").sum()
cols = list(totals.columns[
    totals.columns.str.startswith("PEOPLE") 
    & ~totals.columns.str.startswith("PEOPLE_SNPP")
    & totals.columns.str.contains("od1")
])
cols.append('PEOPLE_SNPP_0-unplanned-od1')
totals = totals[cols]
totals

In [ ]:
totals * 1.1582

In [ ]:
dataset[dataset.GEOGRAPHY_CODE == 'E06000031'][['YEAR', 'PEOPLE_SNPP_0-unplanned-od1', 'PEOPLE_0-unplanned-od1', 'PEOPLE_1-new-cities-od1', 'PEOPLE_2-expansion-od1']].plot(x='YEAR')

In [ ]:
totals.plot()

In [ ]:
arc_all = dataset[dataset.GEOGRAPHY_CODE.isin(arc_lads.geo_code)] \
  .copy() \
  .merge(arc_lads, left_on='GEOGRAPHY_CODE', right_on='geo_code') \
  [[
    'YEAR', 'GEOGRAPHY_CODE', 'geo_label',
    'PEOPLE_SNPP_0-unplanned', 'PEOPLE_0-unplanned-od1', 'PEOPLE_1-new-cities-od1', 'PEOPLE_2-expansion-od1',
    'HOUSEHOLDS_baseline', 'HOUSEHOLDS_0-unplanned', 'HOUSEHOLDS_1-new-cities', 'HOUSEHOLDS_2-expansion',
    'GVA_baseline', 'GVA_0-unplanned', 'GVA_1-new-cities', 'GVA_2-expansion', 
    'JOBS_baseline', 'JOBS_0-unplanned', 'JOBS_1-new-cities', 'JOBS_2-expansion'
]].rename(columns={ 
    'geo_label': 'GEOGRAPHY_NAME',
    'PEOPLE_SNPP_0-unplanned': 'PEOPLE_baseline', 
    'PEOPLE_0-unplanned-od1': 'PEOPLE_0-unplanned', 
    'PEOPLE_1-new-cities-od1': 'PEOPLE_1-new-cities', 
    'PEOPLE_2-expansion-od1': 'PEOPLE_2-expansion',
})
arc_all.HOUSEHOLDS_baseline = arc_all.HOUSEHOLDS_baseline.round().astype(int)
arc_all

In [ ]:
arc_all.columns

In [ ]:
len(arc_lads) == len(arc_all.GEOGRAPHY_CODE.unique())

In [ ]:
len(arc_all) == len(arc_all.GEOGRAPHY_CODE.unique()) * len(arc_all.YEAR.unique())

In [ ]:
arc_all.to_csv('results_for_comparison.csv', index=False)

### Summarise people-per-household

In [ ]:
pph = arc_all \
  .groupby("YEAR").sum() \
  [arc_all.columns[arc_all.columns.str.startswith("PEOPLE") | arc_all.columns.str.startswith("HOUSE")]]

for key in ('baseline', '0-unplanned', '1-new-cities', '2-expansion'):
    # 1. calculate people per household in baseline and model results
    pph["PPH_{}".format(key)] = pph["PEOPLE_{}".format(key)] / pph["HOUSEHOLDS_{}".format(key)]
    
    # 2. calculate expected total population for all-Arc, per-scenario based on baseline 2050 pph
    if key != 'baseline':
        pph['EXP_POP_{}'.format(key)] = pph['HOUSEHOLDS_{}'.format(key)] * pph.PPH_baseline
        pph['PEOPLE_SCALE_FACTOR_{}'.format(key)] = pph['EXP_POP_{}'.format(key)] / pph["PEOPLE_{}".format(key)]
        
columns = pph.columns[pph.columns.str.startswith('EXP_POP') | pph.columns.str.startswith('PEOPLE')]
pph = pph[columns]
pph.tail(1)

In [ ]:
scale_factors = pph[pph.columns[pph.columns.str.startswith('PEOPLE_SCALE')]]
scale_factors.tail(1)

In [ ]:

arc_scale = arc_all.merge(scale_factors, on='YEAR', how='left')
for key in ('0-unplanned', '1-new-cities', '2-expansion'):
    # 3. scale initial results (in Arc area only) up to meet some expectation
    arc_scale['SCALED_PEOPLE_{}'.format(key)] = arc_scale['PEOPLE_{}'.format(key)] \
        * arc_scale['PEOPLE_SCALE_FACTOR_{}'.format(key)] 
    
    # 4. calculate rescaled people-per-household for checking
    arc_scale['SCALED_PEOPLE_PER_HOUSEHOLD_{}'.format(key)] = arc_scale['SCALED_PEOPLE_{}'.format(key)] \
        / arc_scale['HOUSEHOLDS_{}'.format(key)] 
    
arc_scale.head()

In [ ]:
arc_scale_steps = arc_scale[arc_scale.YEAR.isin([2015, 2030, 2050])]
arc_scale_steps.to_csv('scaled_results_for_comparison.csv', index=False)

In [ ]:
arc_scale_steps[
    (arc_scale_steps['SCALED_PEOPLE_PER_HOUSEHOLD_0-unplanned'] > 3) 
    | (arc_scale_steps['SCALED_PEOPLE_PER_HOUSEHOLD_1-new-cities'] > 3)
    | (arc_scale_steps['SCALED_PEOPLE_PER_HOUSEHOLD_2-expansion'] > 3)
]